In [2]:
library(Seurat)
library(ggplot2)
library(dplyr)
library(scales)
library(NICHES)
library(stringr)
library(reshape2)
library(cowplot)

In [ ]:
options(warn=-1)

In [34]:
getwd()

[1] "/Users/pandavis/Desktop/lab_directory/pvat_8w_24w_data/codebase/niches_cci"

In [4]:
list.files('../exported_data/')

[1] "pvat_24weeks_control_female_expr.csv"    
 [2] "pvat_24weeks_control_female_metadata.csv"
 [3] "pvat_24weeks_control_female.h5ad"        
 [4] "pvat_24weeks_control_male_expr.csv"      
 [5] "pvat_24weeks_control_male_metadata.csv"  
 [6] "pvat_24weeks_control_male.h5ad"          
 [7] "pvat_24weeks_hf_female_expr.csv"         
 [8] "pvat_24weeks_hf_female_metadata.csv"     
 [9] "pvat_24weeks_hf_female.h5ad"             
[10] "pvat_24weeks_hf_male_expr.csv"           
[11] "pvat_24weeks_hf_male_metadata.csv"       
[12] "pvat_24weeks_hf_male.h5ad"               
[13] "pvat_8weeks_control_female_expr.csv"     
[14] "pvat_8weeks_control_female_metadata.csv" 
[15] "pvat_8weeks_control_female.h5ad"         
[16] "pvat_8weeks_control_male"                
[17] "pvat_8weeks_control_male_expr.csv"       
[18] "pvat_8weeks_control_male_metadata.csv"   
[19] "pvat_8weeks_control_male.h5ad"           
[20] "pvat_8weeks_hf_female_expr.csv"          
[21] "pvat_8weeks_hf_female_metadata.csv"      
[22] "pvat_8weeks_hf_female.h5ad"              
[23] "pvat_8weeks_hf_male_expr.csv"            
[24] "pvat_8weeks_hf_male_metadata.csv"        
[25] "pvat_8weeks_hf_male.h5ad"

In [ ]:
# Read expr matrix 
#pvat_8w_male_expr_control <- read.csv('../exported_data/pvat_8weeks_control_male_expr.csv', row.names=1)
#pvat_8w_male_metadata_control <- read.csv('../exported_data/pvat_8weeks_control_male_metadata.csv', row.names=1)
#pvat_8w_male_expr_hf <- read.csv('../exported_data/pvat_8weeks_hf_male_expr.csv', row.names=1)
#pvat_8w_male_metadata_hf <- read.csv('../exported_data/pvat_8weeks_hf_male_metadata.csv', row.names=1)

In [35]:
# Read expr matrix 8 weeks
pvat_8w_male_expr_control <- read.csv('exported_unnormalized_data/pvat_8weeks_control_male_expr.csv', row.names=1)
pvat_8w_male_expr_hf <- read.csv('exported_unnormalized_data/pvat_8weeks_hf_male_expr.csv', row.names=1)
pvat_8w_female_expr_control <- read.csv('exported_unnormalized_data/pvat_8weeks_control_female_expr.csv', row.names=1)
pvat_8w_female_expr_hf <- read.csv('exported_unnormalized_data/pvat_8weeks_hf_female_expr.csv', row.names=1)

In [36]:
# Read metadata 8 weeks
pvat_8w_male_metadata_control <- read.csv('exported_unnormalized_data/pvat_8weeks_control_male_metadata.csv', row.names=1)
pvat_8w_male_metadata_hf <- read.csv('exported_unnormalized_data/pvat_8weeks_hf_male_metadata.csv', row.names=1)
pvat_8w_female_metadata_control <- read.csv('exported_unnormalized_data/pvat_8weeks_control_female_metadata.csv', row.names=1)
pvat_8w_female_metadata_hf <- read.csv('exported_unnormalized_data/pvat_8weeks_hf_female_metadata.csv', row.names=1)

In [37]:
# Read expr matrix 24 weeks
pvat_24w_male_expr_control <- read.csv('exported_unnormalized_data/pvat_24weeks_control_male_expr.csv', row.names=1)
pvat_24w_male_expr_hf <- read.csv('exported_unnormalized_data/pvat_24weeks_hf_male_expr.csv', row.names=1)
pvat_24w_female_expr_control <- read.csv('exported_unnormalized_data/pvat_24weeks_control_female_expr.csv', row.names=1)
pvat_24w_female_expr_hf <- read.csv('exported_unnormalized_data/pvat_24weeks_hf_female_expr.csv', row.names=1)

In [38]:
# Read metadata 24 weeks
pvat_24w_male_metadata_control <- read.csv('exported_unnormalized_data/pvat_24weeks_control_male_metadata.csv', row.names=1)
pvat_24w_male_metadata_hf <- read.csv('exported_unnormalized_data/pvat_24weeks_hf_male_metadata.csv', row.names=1)
pvat_24w_female_metadata_control <- read.csv('exported_unnormalized_data/pvat_24weeks_control_female_metadata.csv', row.names=1)
pvat_24w_female_metadata_hf <- read.csv('exported_unnormalized_data/pvat_24weeks_hf_female_metadata.csv', row.names=1)

In [39]:
process_seurat_obj <- function(expr_control, expr_hf, metadata_control, metadata_hf, label, group) {
    
  combined_expr <- rbind(expr_control, expr_hf)
  combined_metadata <- rbind(metadata_control, metadata_hf)
  
  seurat_obj <- CreateSeuratObject(counts = t(combined_expr), meta.data = combined_metadata)
  #seurat_obj@assays$RNA@layers$data <- seurat_obj@assays$RNA@layers$counts
  
  seurat_obj <- NormalizeData(seurat_obj)
  seurat_obj <- ScaleData(seurat_obj, assay = "RNA", layer = "data")
  seurat_obj <- FindVariableFeatures(seurat_obj)
  
  seurat_obj <- RunPCA(seurat_obj)
  seurat_obj <- RunUMAP(seurat_obj, dims = 1:10)
  
  seurat_obj <- SetIdent(seurat_obj, value = "celltype_broad")
  
  file_name <- paste0("combined_", label, "_", group, ".rds")
  saveRDS(seurat_obj, file = file_name)
  
  return(seurat_obj)
}

In [40]:
combined_male_8weeks <- process_seurat_obj(
  expr_control = pvat_8w_male_expr_control,
  expr_hf = pvat_8w_male_expr_hf,
  metadata_control = pvat_8w_male_metadata_control,
  metadata_hf = pvat_8w_male_metadata_hf,
  label = "male",
  group = "8weeks"
)

Warning message:
“Feature names cannot have underscores ('_'), replacing with dashes ('-')”
Warning message:
“Data is of class matrix. Coercing to dgCMatrix.”
Normalizing layer: counts

Centering and scaling data matrix

Finding variable features for layer counts

PC_ 1 
Positive:  Gk, Ppargc1a, Pde3b, Ntrk3, Adra1a, Efna5, Gpd2, Gpd1, Ptprt, Lpin1 
	   Pdk4, Ddhd1, Acaca, Cables1, Pc, Tbc1d4, Chka, Gmpr, Ucp1, ENSRNOG00000064615 
	   Slc4a7, Chpt1, Ppara, Fam13a, Slc22a3, Pck1, Pde3a, Pank1, Rnf150, Prkaa2 
Negative:  Mast4, Dst, Tshz2, Prkg1, Cald1, Plxdc2, Rhoj, Bicc1, Col5a2, Stard13 
	   Auts2, Akt3, Zfp521, Col3a1, Lama2, Apbb2, Abi3bp, Ldb2, Naaladl2, Arhgap6 
	   Grb10, Fbn1, Sash1, Fyn, Zfp385d, Syne1, Nhsl1, Gulp1, Kalrn, Col6a2 
PC_ 2 
Positive:  Ptprb, Cdh13, Shank3, Flt1, Egfl7, Pecam1, St6galnac3, Etl4, Arhgap31, Cyyr1 
	   Prkch, Emcn, Adgrf5, Eng, LOC108352070, Galnt18, Ccdc85a, Erg, Adgrl4, Podxl 
	   Nos3, Dach1, Elmo1, Cdh5, Arap3, Ppp1r16b, Kdr, Tek, Stox2, Ablim3 


In [41]:
combined_female_8weeks <- process_seurat_obj(
  expr_control = pvat_8w_female_expr_control,
  expr_hf = pvat_8w_female_expr_hf,
  metadata_control = pvat_8w_female_metadata_control,
  metadata_hf = pvat_8w_female_metadata_hf,
  label = "female",
  group = "8weeks"
)

Warning message:
“Feature names cannot have underscores ('_'), replacing with dashes ('-')”
Warning message:
“Data is of class matrix. Coercing to dgCMatrix.”
Normalizing layer: counts

Centering and scaling data matrix

Finding variable features for layer counts

PC_ 1 
Positive:  Gpd2, Pank1, Lpin1, Ptprt, Gmpr, Cables1, Acaca, Ppargc1a, Pnpla3, Ppara 
	   Ddhd1, Cbfa2t3, Pck1, Ntrk3, Acly, Dlat, Impa2, Tbc1d4, Npr3, Elovl6 
	   Cpeb3, Acss2, Rnf150, Slc22a3, Gpam, Rgs7, Rnf152, Tspan18, Sytl5, Pfkfb3 
Negative:  Mast4, Dst, Bicc1, Prkg1, Cald1, Tshz2, Stard13, Grb10, Fbxl7, Col3a1 
	   Abi3bp, Zfp521, Col5a2, Apbb2, Auts2, Rhoj, Nav1, Akt3, Lama2, Flnb 
	   Plxdc2, Naaladl2, Plpp3, Hip1, Eya2, Egfr, Fyn, Rftn1, Ror1, Phactr2 
PC_ 2 
Positive:  Col3a1, Rbms3, Egfr, Lama2, Col6a2, Zfp385d, Gli2, Naaladl2, Zfpm2, Ror1 
	   Col6a1, Ddr2, Lsamp, Col5a2, Prkg1, Abca8a, Adamts2, Col5a3, Col1a2, Adamtsl1 
	   Col6a3, Slit3, Errfi1, Rcan2, Dennd2a, Medag, Gpc6, Cpq, Sema3c, Magi2 
Negative: 

In [42]:
combined_male_24weeks <- process_seurat_obj(
  expr_control = pvat_24w_male_expr_control,
  expr_hf = pvat_24w_male_expr_hf,
  metadata_control = pvat_24w_male_metadata_control,
  metadata_hf = pvat_24w_male_metadata_hf,
  label = "male",
  group = "24weeks"
)

Warning message:
“Feature names cannot have underscores ('_'), replacing with dashes ('-')”
Warning message:
“Data is of class matrix. Coercing to dgCMatrix.”
Normalizing layer: counts

Centering and scaling data matrix

Finding variable features for layer counts

PC_ 1 
Positive:  Lpin1, Ppara, Ucp1, Pank1, Acaca, Slc4a4, Acly, Kcnb1, Pdk4, Elovl6 
	   Pnpla3, Lpl, Rgs7, Ppargc1a, Gk, Lrp6, Cables1, Septin9, Gpam, Stimate 
	   Rnf150, Acss2, Ddhd1, Fasn, Pfkfb3, Enpp2, Impa2, Npr3, Sgcd, Pfkfb4 
Negative:  Cdh13, Fbxl7, Etl4, Flt1, Adgrf5, Dach1, Arhgap31, Fli1, Syne1, Cyyr1 
	   Mast4, Rftn1, Ptprb, Galnt18, Shank3, Eng, Apbb2, Stard13, Ccdc85a, Tshz2 
	   Mef2c, Prkch, LOC108352070, Akt3, Meox2, Lmo7, Kitlg, Kalrn, Flnb, St6galnac3 
PC_ 2 
Positive:  Zfp385d, Lama2, Prkg1, Plxdc2, Abcc9, Kcnq5, Rbms3, Col6a6, Col6a2, Kcnip4 
	   Gli2, Ddr2, Dennd2a, Gucy1a2, Naaladl2, Abca8a, Col5a2, AABR07044388.2, Col6a3, Cacna1c 
	   Ror1, Col6a1, Arhgap6, Egfr, Prrx1, Gli3, Irag1, Pdgfrb, Antxr1

In [43]:
combined_female_24weeks <- process_seurat_obj(
  expr_control = pvat_24w_female_expr_control,
  expr_hf = pvat_24w_female_expr_hf,
  metadata_control = pvat_24w_female_metadata_control,
  metadata_hf = pvat_24w_female_metadata_hf,
  label = "female",
  group = "24weeks"
)

Warning message:
“Feature names cannot have underscores ('_'), replacing with dashes ('-')”
Warning message:
“Data is of class matrix. Coercing to dgCMatrix.”
Normalizing layer: counts

Centering and scaling data matrix

Finding variable features for layer counts

PC_ 1 
Positive:  Cdh13, Dst, Arhgap31, Adgrf5, Syne1, Flt1, Shank3, Cyyr1, Rftn1, Prkch 
	   Dach1, Fbxl7, Eng, Kalrn, Pecam1, Lmo7, Mef2c, Apbb2, Galnt18, Ccdc85a 
	   Stox2, Egfl7, Magi1, Mcc, Kitlg, Tshz2, Stard13, Flnb, Tcf4, Meox2 
Negative:  Slc4a4, Efna5, Pde3b, ENSRNOG00000064615, Pank1, Gk, Ppara, Pdk4, Ppargc1a, Pde3a 
	   Acaca, Ptprt, Lpin1, Ddhd1, Esrrg, Chpt1, Rgs7, Pfkfb3, Cables1, Agmo 
	   Enpp2, Gmpr, Elovl6, Sh2b2, Sgcd, Zbtb7c, Gpam, Dio2, Fam13a, Aspg 
PC_ 2 
Positive:  Lama2, Prkg1, Plxdc2, Kcnip4, Rbms3, Abcc9, Col6a2, Col6a6, Col3a1, Gli2 
	   Zfp385d, Ddr2, Col5a2, Gucy1a2, Abca8a, AABR07044388.2, Dennd2a, Kcnq5, Col6a1, Naaladl2 
	   Col5a3, Egfr, Irag1, Cacna1c, Fbn1, Gpc6, Prrx1, Pdgfrb, Nhs, Errf